In [1]:
from datetime import datetime
import pandas as pd
import numpy as np

In [2]:
f = open("fvtt-log-Mon-Jun-13-2022.txt",encoding="utf8")
data = f.read()
data_list = data.split("---------------------------")
clean = []
characters = []
try:
    for i, item in enumerate(data_list):
        try:
            temp_list = item.split("\n")
            temp_list = list(filter(None, temp_list))
            roll_result = temp_list[1]
            found_action = temp_list[2].find("<strong>")
            try:
                if found_action < 0:
                    found_action = temp_list[2].find('<h4 class="action">')
                    if found_action < 0:
                        found_action = -1
                    else:
                        historicType = '<h4 class="action">'
                else:
                    historicType = "<strong>"
            except:
                found_action = -1
        except:
            found_action = -1
        if roll_result.isnumeric() and found_action>-1:
            dateString = temp_list[0][1:temp_list[0].index(",")]
            date = datetime.strptime(dateString,'%m/%d/%Y')
            character = temp_list[0][temp_list[0].index("]")+2:]
            if character not in characters:
                characters.append(character)
            if historicType == "<strong>":
                action = temp_list[2][temp_list[2].index("<strong>")+8:temp_list[2].index("</strong>")]
            elif historicType == '<h4 class="action">':
                action = temp_list[2][temp_list[2].index('<h4 class="action">')+19:temp_list[2].index("</h4")]
            d20roll = temp_list[-1][temp_list[-1].index("=")+1:temp_list[-1].index("=")+4].strip()
            clean.append([character,roll_result,action,d20roll,date])
        data_list[i] = temp_list
    df = pd.DataFrame(clean, columns= ["Character", "Roll Result","Action","D20 Roll", "Date"])
except Exception as e:
    print(dateString)
    print(date)
    print(data_list[i])
    print("error in line " + str(i))
    print(e)


In [3]:
df
dfStrike = df

In [4]:
#df.to_csv('df.csv')

In [5]:
#data_list

In [6]:
import matplotlib.pyplot as plt
import statistics

In [ ]:
summary = []
sinceWhen = datetime(2022,6,12)
x_dice = []
searchString = ""
for player in characters:
    historic = []
    x_dice = []
    rolls = pd.DataFrame(data ={"Rolls":[0]*20})
    rolls.index = rolls.index+1
    newdf = df[df["Character"] == player]
    newdf = newdf[newdf["Date"] >= sinceWhen]
    newdf["D20 Roll"] = pd.to_numeric(newdf["D20 Roll"])
    newdf = newdf.loc[newdf['Action'].str.contains(searchString, regex=False)]
    if len(newdf.index) > 0:
        plt.figure()
        counts = newdf['D20 Roll'].value_counts()
        counts = newdf['D20 Roll'].value_counts()
        counts = counts.to_frame()
        counts.index = pd.to_numeric(counts.index)
        counts = counts.sort_index()
        newRolls = pd.concat([rolls, counts],axis=1)
        newRolls = newRolls.fillna(value={"D20 Roll":0})
        newRolls['D20 Roll'].plot.bar()
        plt.xlabel('Roll')
        plt.title(player + " Mean: " + str(newdf['D20 Roll'].mean()))
        plt.savefig(player + ".jpg", dpi=72)
        summary.append([newdf['D20 Roll'].mean(),player,len(newdf.index)])   
summary.sort()

In [8]:
todaydf = df[df["Date"] > sinceWhen]
todaydf

,Character,Roll Result,Action,D20 Roll,Date
7036,A.R.C.S. Type B,36,Strike: Unarmed Attack,18,2022-05-30
7037,Schuldiner,30,Perception Check,13,2022-05-30
7038,A.R.C.S. Type B,28,Skill Check: Crafting,2,2022-05-30
7039,A.R.C.S. Type B,34,Skill Check: Crafting,8,2022-05-30
7040,Stone Golem,21,Initiative: Perception,2,2022-05-30
...,...,...,...,...,...
7248,Pnigalion,27,Strike: Shock Siccatite Hatchet,6,2022-06-06
7249,Humanoid Xulgath 5,21,Skill Check: Stealth,5,2022-06-06
7250,Humanoid Xulgath 5,33,Skill Check: Stealth,17,2022-06-06
7251,Groa,25,Perception Check,5,2022-06-06


In [9]:
summary.reverse()
summary

[[14.666666666666666, 'Humanoid Xulgath 82', 3],
 [12.416666666666666, 'Schuldiner', 12],
 [11.714285714285714, 'Humanoid Xulgath 5', 7],
 [11.4, 'A.R.C.S. Type B', 10],
 [11.3125, 'Colaag', 16],
 [11.214285714285714, 'Monkey D. Kaido', 14],
 [11.055555555555555, 'Pnigalion', 36],
 [10.666666666666666, 'Giant Flytrap', 9],
 [10.5, 'Booga', 4],
 [10.434782608695652, 'Groa', 23],
 [10.333333333333334, 'Animal 29', 3],
 [10.25, 'Hooklimb Xulgath', 16],
 [9.818181818181818, 'Roger D. Gold', 11],
 [9.636363636363637, 'Mindless Plant 73', 11],
 [9.454545454545455, 'Stone Golem', 11],
 [9.133333333333333, 'Xulgath Herd-Tender', 15],
 [8.0, 'Aerith', 1],
 [7.571428571428571, 'Ooga', 7],
 [6.0, 'Animal 83', 1],
 [5.333333333333333, 'Animal 97', 3],
 [5.0, 'Roger', 1],
 [4.0, 'Mindless Plant 54', 2],
 [2.0, 'Bulette', 1]]